In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # regular expression libary.
import nltk # Natural Language toolkit
nltk.download("stopwords")  #downloading stopwords
nltk.download('punkt')
from nltk import word_tokenize,sent_tokenize
nltk.download('wordnet')
import nltk as nlp
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from sklearn.feature_extraction.text import CountVectorizer 
from wordcloud import WordCloud 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,precision_score
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer #Bag of Words
import seaborn as sns 
import collections 
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


In [ ]:
file_name="route"
df = pd.read_excel(file_name, sheet_name=None)

In [ ]:
df_copy=df.copy()
df_copy["YL"].head()

In [ ]:
Encoder = preprocessing.LabelEncoder()
y_train=Encoder.fit_transform(df_copy["Ek2"][:1400].categoryText)
y_test=Encoder.fit_transform(df_copy["Ek2"][1400:].categoryText)

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt

data =df_copy["YL"].Category.value_counts()
sns.barplot(y = data.index, x = data, orient='h')
#plt.savefig('category.eps', format='eps')

In [ ]:
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('omw-1.4')

In [ ]:
def clearText(textList):
    text_list=[]

    for text in textList:
        text = text.lower()  #Büyük harften -Küçük harfe çevirme
        text = re.sub("[^abcçdefgğhıijklmnoöprsştuüvyz]"," ",text)
        text=nlp.word_tokenize(text) # splits the words that are in the sentence from each other.
        text =[word for word in text if not word in set(stopwords.words("turkish"))]
        lemma=nlp.WordNetLemmatizer()
        text=[lemma.lemmatize(word) for word in text] # this code finds the root of the word for a word in the sentence and change them to their root form.
        text=" ".join(text)
        text_list.append(text) # store sentences in list

    return text_list

text_list=clearText(df_copy["YL"].text)
train_text_list=clearText(df_copy["Ek2"][:1400].text)
test_text_list=clearText(df_copy["Ek2"][1400:].text)

In [ ]:
max_features=500 # "number" most common(used) words in reviews
count_vectorizer=CountVectorizer(max_features=max_features) 
X_train=count_vectorizer.fit_transform(train_text_list).toarray()
X_test=count_vectorizer.fit_transform(test_text_list).toarray()

In [ ]:
X_train=train_text_list
X_test=test_text_list

In [ ]:
import tensorflow as tf
max_lenght=100


tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)


word_index = tokenizer.word_index # creating word dict for words in training

sequences = tokenizer.texts_to_sequences(X_train)  # replacing words with the number corresponding to them in the dictionary(word_index)

X_train_padded = pad_sequences(sequences, padding='post',maxlen=max_lenght) # padding words

print(len(word_index)) # I have 94836 words in my dictionary

In [ ]:
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences,padding="post",maxlen=max_lenght)

In [ ]:
vocab_size = len(tokenizer.word_index)+1
embedding_dim=16

model = tf.keras.Sequential([
    
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=100),
    
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(3, activation='softmax')
])

adam= tf.keras.optimizers.Adam(lr=0.01) 


model.compile(loss='sparse_categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model.summary()

In [ ]:
y_train_label=np.asarray(y_train).reshape(-1,1)
y_test_label=np.asarray(y_test).reshape(-1,1)
#y_train_label = np.asarray(y_train).reshape((-1,1))
#y_test_label = np.asarray(y_test).reshape((-1,1))

In [ ]:
num_epochs = 10
BATCH_SIZE=64
history=model.fit(X_train_padded,y_train_label,batch_size=BATCH_SIZE ,epochs=num_epochs, validation_data=(X_test_padded,y_test_label))

In [ ]:
plt.plot(history.history["accuracy"],color="green")
plt.plot(history.history["loss"],color="red")
plt.title("Train accuracy and Train loss")
plt.legend(["Accuracy","Loss"])
plt.grid()
plt.savefig('lstmtektrain.eps', format='eps')

In [ ]:
plt.plot(history.history["val_accuracy"],color="blue")
plt.plot(history.history["val_loss"],color="orange")
plt.title("Test accuracy and Test loss")
plt.legend(["Val_accuracy","Val_loss"])
plt.grid()
plt.savefig('lsttek.eps', format='eps')

In [ ]:
print("Accuracy of the model on Training Data is - " , model.evaluate(X_train_padded,y_train_label)[1]*100 , "%")
print("Accuracy of the model on Testing Data is - " , model.evaluate(X_test_padded,y_test_label)[1]*100 , "%")